In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [28]:
############Leemos el CSV######################
df = pd.read_csv('poker-hand-tiny.csv')

############Limpiamos el CSV###################
for i in range(5,10):
    #Creamos lista con los indices a eliminar
    idmemor = df[df['Clase']== i].index
    #Eliminamos los indices del csv
    df = df.drop(idmemor)

#Define cada dataset
y_train = df.iloc[:,10]
x_train = df.iloc[:,0:10]

In [29]:
###################### CREACIÓN DATOS SINTÉTICOS #####################
smote = SMOTE()
X_train, Y_train = smote.fit_sample(x_train, y_train)
print(X_train, Y_train)

        S1  C1  S2  C2  S3  C3  S4  C4  S5  C5
0        1   1   1  13   2   4   2   3   1  12
1        3  12   3   2   3  11   4   5   2   5
2        1   9   4   6   1   4   3   2   3   9
3        1   4   3  13   2  13   2   1   3   6
4        3  10   2   7   1   2   2  11   4   9
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
751440   3   4   1   6   3   6   3   4   1   7
751441   3   7   1   7   4  10   3   6   1   8
751442   4   9   1   8   1  11   2  10   2   7
751443   3   2   1   4   3   2   1   5   2   6
751444   2   2   4   3   1   4   1   3   2   6

[751445 rows x 10 columns] 0         0
1         1
2         1
3         1
4         0
         ..
751440    4
751441    4
751442    4
751443    4
751444    4
Name: Clase, Length: 751445, dtype: int64


In [30]:
############# ÁRBOL DE DECISIÓN #################
arboldecision = DecisionTreeClassifier()
parametros = {'criterion': ['gini', 'entropy'],'splitter':['best', 'random'],'max_depth': [1,4,7,8,9,10]}

In [31]:
############# VALIDACIÓN CRUZADA - Grid Search ##############
clf = GridSearchCV(arboldecision, parametros, cv = 3)
clf.fit(X_train, Y_train)
clf.cv_results_
df = pd.DataFrame(clf.cv_results_)
print(df[['param_criterion', 'param_max_depth', 'param_splitter', 'mean_test_score', 'rank_test_score' ]])

   param_criterion param_max_depth param_splitter  mean_test_score  \
0             gini               1           best         0.225785   
1             gini               1         random         0.220520   
2             gini               2           best         0.239586   
3             gini               2         random         0.239094   
4             gini               3           best         0.253607   
5             gini               3         random         0.248638   
6             gini               4           best         0.259895   
7             gini               4         random         0.255251   
8             gini               5           best         0.276884   
9             gini               5         random         0.268449   
10            gini               6           best         0.293791   
11            gini               6         random         0.294576   
12            gini               7           best         0.319104   
13            gini  

In [1]:
######## NAIVE BAYES ##########
bayes = MultinomialNB()
val_cruzada = cross_validate(bayes, X_train, Y_train, cv = 3)
print(val_cruzada.keys())
print('Resulatado Naive Bayes: ', val_cruzada['test_score']*100)

NameError: name 'MultinomialNB' is not defined

In [38]:
model = clf.best_estimator_
print('Resultado Arbol de Decisión:' , model.score(X_train,Y_train)*100)


41.16548782678706


In [23]:



############ Cuento componentes del árbol ###############

n_nodes = model.tree_.node_count
children_left = model.tree_.children_left
children_right = model.tree_.children_right
feature = model.tree_.feature
threshold = model.tree_.threshold

node_indicator = model.decision_path(X_train)

leave_id = model.apply(X_train)

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        print(X_test[sample_id, feature[node_id]])
        
        threshold_sign = "<="
    else:
        print(X_test[sample_id, feature[node_id]])
        print(sample_id)
        print(feature[node_id])
        print(X_test)
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

ValueError: Number of features of the model must match the input. Model n_features is 10 and input n_features is 4 